In [ ]:
# necessary imports
import numpy as np
import numpy.random as rng 
import tensorflow as tf
import matplotlib.pyplot as plt 
import sklearn.metrics
import keras.layers as layers
import keras.models
import scipy.stats
import keras.utils
import json
import pandas as pd

In [ ]:
with open('cars_data.json', 'r') as f:
    cars = json.load(f)
df = pd.DataFrame(cars)

KM_PER_LITER_PER_MILES_PER_GALLON = 0.425144

df['kpl'] = df['Miles_per_Gallon'] * KM_PER_LITER_PER_MILES_PER_GALLON
df['weight'] = df['Weight_in_lbs'] * 0.453592
df = df[~np.isnan(df['kpl']) & ~np.isnan(df['Horsepower'])]

In [ ]:
X = np.array(df['Horsepower'])[:,np.newaxis]
y = np.array(df['kpl'])[:,np.newaxis]

N_HIDDEN = 3

input_node = layers.Input(shape=(X.shape[1],))
hidden_layer = layers.Dense(N_HIDDEN, activation='relu')(input_node)
output_node = layers.Dense(1, activation='linear')(hidden_layer)
model = keras.models.Model(inputs=input_node, outputs=output_node)
model.compile('sgd', 'mean_squared_error')


In [ ]:
# train the model
Xnormed = (X - np.mean(X)) / np.std(X)
ynormed = np.log(y)
h = model.fit(x = Xnormed, y=ynormed, verbose=False, epochs=1000, batch_size=X.shape[0])

In [ ]:
# f, ax = plt.subplots(1, 1, figsize=(20, 10))
# ax.plot(h.history['loss'], linewidth=5)
# ax.plot(h.history['val_loss'], linewidth=5)
# ax.set_title('Model Loss')
# ax.set_ylabel('Loss')
# ax.set_xlabel('Epoch')
# ax.legend(['Train', 'Test'], loc='upper left')

In [ ]:
hps = np.linspace(0, 500, 1000)[:,np.newaxis]
log_kpls = model.predict((hps - np.mean(X))/np.std(X))

mse = np.mean(np.square(np.exp(model.predict(Xnormed)) - y))
print("MSE: %f" % mse)
f, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(df['Horsepower'], df['kpl'], marker='o', linestyle='', markersize=10, color='orange', markeredgecolor='black')

ax.plot(hps, np.exp(log_kpls), linewidth=5, linestyle='--')
ax.tick_params(axis='x', labelsize=28)
ax.tick_params(axis='y', labelsize=28)
ax.set_ylabel('Kilometers per Liter', fontsize=32, fontweight='bold')
ax.set_xlabel('Horsepower', fontsize=32, fontweight='bold')
ax.grid('both')
ax.set_ylim([0, 21])
ax.legend(fontsize=24, frameon=False)
plt.savefig('../tmp/nn_fit.png', bbox_inches='tight')
